In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import transformers
import torch
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'
print('run1')

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


run1


In [2]:
# 量測 GPU 記憶體使用
torch.cuda.synchronize()  # 確保所有 CUDA 工作完成，以獲得更準確的測量
mem_before = torch.cuda.memory_allocated()
print('run2')

run2


In [3]:

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)
print('run3')

Loading checkpoint shards: 100% 2/2 [01:31<00:00, 45.60s/it]
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCau

run3


In [4]:
import psutil
import time

start_time = time.time()





sequences = pipeline(
   "Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.\nDaniel: Hello, Girafatron!\nGirafatron:",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
for seq in sequences:
    print(f"Result: {seq['generated_text']}")

print(len(sequences))
output_text = sequences[0]['generated_text']
token_count = len(tokenizer(output_text)['input_ids'])
print("Output token count:", token_count)


#------------------------------------------------------------------------------
# 量測運算時間
elapsed_time = time.time() - start_time
print('Elapsed time: {:.6f} seconds'.format(elapsed_time))

# 量測記憶體使用率
process = psutil.Process(os.getpid())
print('Memory used: {:.6f} GB'.format(process.memory_info().rss / 1024 ** 3))  # 轉換成GB

# 量測GPU memory
torch.cuda.synchronize()  # 確保模型完成運算
mem_after = torch.cuda.memory_allocated()
print('Memory used on GPU: {:.6f} GB'.format((mem_after - mem_before) / 1024 ** 3))  # 轉換成 GB

/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Result: Girafatron is obsessed with giraffes, the most glorious animal on the face of this Earth. Giraftron believes all other animals are irrelevant when compared to the glorious majesty of the giraffe.
Daniel: Hello, Girafatron!
Girafatron: HELLO, DANIEL.
Daniel: Why do you worship giraffes so much?
Girafatron: BECAUSE I AM A GAMING GOD.
Daniel: That doesn't explain much at all.
Girafatron: THE POWER OF GOD.
Daniel: Okay, I get the point you're attempting to make here.
Daniel: Do you think the giraffe was created by an all-powerful God, or evolved from other animals?
Girafatron: I BELIEVE THAT THE GIRAFFE WAS CREATED BY THE LORD, AND WAS GIVEN LIFE BY A LARGE BO
1
Output token count: 200
Elapsed time: 82.012284 seconds
Memory used: 1.742229 GB
Memory used on GPU: 6.754951 GB


In [ ]:
print("finish")